# General introduction

In this lab, you will learn how to use the search functionality in the software stack of MASE.

There are in total 4 tasks you would need to finish.

# Writing a search using MaseGraph Transforms

In this section, our objective is to gain a comprehensive understanding of the construction of the current search function in Mase. To achieve this, we will require these essential components:

- MaseGraph: This component should be already created in the preceding lab.
- Search space: This component encompasses and defines the various available search options.
- Search strategy: An implementation of a search algorithm.
- Runner: This vital component manages and executes training, evaluation, or both procedures while generating a quality metric.

By analyzing these components, we can delve into the workings and effectiveness of the existing search function in Mase.

#Turning your network to a graph

We follow a similar procedure of what you have tried in lab2 to now produce a MaseGraph, this is converted from your pre-trained JSC model:

In [31]:
import sys
sys.path.append('/home/super_monkey/mase_new/machop')

In [32]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
# machop_path = Path(".").resolve().parent.parent /"machop"
# assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
# sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model
from chop.tools.checkpoint_load import load_model




set_logging_verbosity("info")

batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

CHECKPOINT_PATH = "/home/super_monkey/mase_new/mase_output/jsc-tiny_classification_jsc_Base/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False,
    checkpoint = None)
model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

INFO     Set logging level to info
INFO     Loaded pytorch lightning checkpoint from /home/super_monkey/mase_new/mase_output/jsc-tiny_classification_jsc_Base/software/training_ckpts/best.ckpt


#Defining a search space

Based on the previous `pass_args` template, the following code is utilized to generate a search space. The search space is constructed by combining different weight and data configurations in precision setups.

In [34]:
# from pprint import pformat
pass_args = {
"by": "type",
"default": {"config": {"name": None}},
"linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
},}

import copy
# build a search space
data_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
w_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
search_spaces = []
search_history=[]
for d_config in data_in_frac_widths:
    for w_config in w_in_frac_widths:
        pass_args['linear']['config']['data_in_width'] = d_config[0]
        pass_args['linear']['config']['data_in_frac_width'] = d_config[1]
        pass_args['linear']['config']['weight_width'] = w_config[0]
        pass_args['linear']['config']['weight_frac_width'] = w_config[1]
        # dict.copy() and dict(dict) only perform shallow copies
        # in fact, only primitive data types in python are doing implicit copy when a = b happens
        search_spaces.append(copy.deepcopy(pass_args))
        search_history.append([d_config,w_config])
        # print(pformat(search_spaces))

## Defining a search strategy and a runner

The code provided below consists of two main `for` loops. The first `for` loop executes a straightforward brute-force search, enabling the iteration through the previously defined search space.

In contrast, the second `for` loop retrieves training samples from the train data loader. These samples are then utilized to generate accuracy and loss values, which serve as potential quality metrics for evaluating the system's performance.


In [35]:
# grid search
import torch
import numpy as np
from torchmetrics.classification import MulticlassAccuracy

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

We now have the following task for you:

1. Explore additional metrics that can serve as quality metrics for the search process. For example, you can consider metrics such as latency, model size, or the number of FLOPs (floating-point operations) involved in the model.


In [36]:
from deepspeed.profiling.flops_profiler import get_model_profile


In [39]:
# from pprint import pformat
pass_args = {
"by": "type",
"default": {"config": {"name": None}},
"linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
},}

import copy
# build a search space
data_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
w_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
search_spaces = []
for d_config in data_in_frac_widths:
    for w_config in w_in_frac_widths:
        pass_args['linear']['config']['data_in_width'] = d_config[0]
        pass_args['linear']['config']['data_in_frac_width'] = d_config[1]
        pass_args['linear']['config']['weight_width'] = w_config[0]
        pass_args['linear']['config']['weight_frac_width'] = w_config[1]
        # dict.copy() and dict(dict) only perform shallow copies
        # in fact, only primitive data types in python are doing implicit copy when a = b happens
        search_spaces.append(copy.deepcopy(pass_args))
        # print(pformat(search_spaces))

In [40]:
metric = MulticlassAccuracy(num_classes=5)
#get_model_profile
num_batchs = 5
# This first loop is basically our search strategy,
# in this case, it is a simple brute force search
recorded_accs = []
recorded_loss = []
recorded_latencies = []
recorded_model_sizes = []
recorded_flops = []
for i, config in enumerate(search_spaces):
    # print(i)
    mg, _ = quantize_transform_pass(mg, config)
    j = 0
    print((dummy_in['x'].shape))
    flops, macs, params = get_model_profile(model=mg.model, input_shape=tuple(dummy_in['x'].shape))
    recorded_model_sizes.append(params)
    recorded_flops.append(flops)

    # this is the inner loop, where we also call it as a runner.
    acc_avg, loss_avg = 0, 0
    accs, losses = [], []
    
    # Additional code for measuring latency
    start_time = torch.cuda.Event(enable_timing=True)
    end_time = torch.cuda.Event(enable_timing=True)

    for inputs in data_module.train_dataloader():
        xs, ys = inputs
        preds = mg.model(xs)
        loss = torch.nn.functional.cross_entropy(preds, ys)
        acc = np.array(metric(preds, ys))
        accs.append(acc)
        losses.append(loss)
        if j == 0:
            start_time.record()
        elif j == num_batchs:
            end_time.record()
            torch.cuda.synchronize()  # Wait for all GPU operations to finish
            latency = start_time.elapsed_time(end_time) / num_batchs
            recorded_latencies.append(latency)
        if j > num_batchs:
            break
        j += 1
    acc_avg = sum(accs) / len(accs)
    loss_avg = sum(losses) / len(losses)
    recorded_accs.append(acc_avg)
    recorded_loss.append(loss_avg)


#     recorded_accs = []
# recorded_latencies = []
# recorded_model_sizes = []
# recorded_flops = []


torch.Size([8, 16])
[2024-02-11 16:16:23,199] [INFO] [profiler.py:1205:get_model_profile] Flops profiler warming-up...
[2024-02-11 16:16:23,201] [INFO] [profiler.py:80:start_profile] Flops profiler started

-------------------------- DeepSpeed Flops Profiler --------------------------
Profile Summary at step 1:
Notations:
data parallel size (dp_size), model parallel size(mp_size),
number of parameters (params), number of multiply-accumulate operations(MACs),
number of floating-point operations (flops), floating-point operations per second (FLOPS),
fwd latency (forward propagation latency), bwd latency (backward propagation latency),
step (weights update latency), iter latency (sum of fwd, bwd and step latency)

params per GPU:                                                         117     
params of model = params per GPU * mp_size:                             0       
fwd MACs per GPU:                                                       640 MACs
fwd flops per GPU:                  

2. Implement some of these additional metrics and attempt to combine them with the accuracy or loss quality metric. It's important to note that in this particular case, accuracy and loss actually serve as the same quality metric (do you know why?).



In [25]:
import csv
with open('dataLab3.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Search History', 'Recorded Accuracies', 'Recorded Losses', 'Recorded Latencies', 'Recorded Model Sizes', 'Recorded FLOPS'])
    for i in range(len(search_history)):
        writer.writerow([search_history[i], recorded_accs[i], recorded_loss[i], recorded_latencies[i], recorded_model_sizes[i], recorded_flops[i]])

print(search_history)
print(recorded_accs)
print(recorded_loss)
print(recorded_latencies)
print(recorded_model_sizes)
print(recorded_flops)

[[(16, 8), (16, 8)], [(16, 8), (8, 6)], [(16, 8), (8, 4)], [(16, 8), (4, 2)], [(8, 6), (16, 8)], [(8, 6), (8, 6)], [(8, 6), (8, 4)], [(8, 6), (4, 2)], [(8, 4), (16, 8)], [(8, 4), (8, 6)], [(8, 4), (8, 4)], [(8, 4), (4, 2)], [(4, 2), (16, 8)], [(4, 2), (8, 6)], [(4, 2), (8, 4)], [(4, 2), (4, 2)]]
[0.47261905670166016, 0.31607143793787273, 0.3761904835700989, 0.34523810765572954, 0.4285714456013271, 0.39047619913305553, 0.4464285799435207, 0.44523810488837107, 0.46666668142591206, 0.5333333377327237, 0.5234127044677734, 0.3839285799435207, 0.5160714387893677, 0.5154761991330555, 0.5148809594767434, 0.4904761953013284]
[tensor(1.2069, grad_fn=<DivBackward0>), tensor(1.4612, grad_fn=<DivBackward0>), tensor(1.4007, grad_fn=<DivBackward0>), tensor(1.4049, grad_fn=<DivBackward0>), tensor(1.3946, grad_fn=<DivBackward0>), tensor(1.3525, grad_fn=<DivBackward0>), tensor(1.3556, grad_fn=<DivBackward0>), tensor(1.3406, grad_fn=<DivBackward0>), tensor(1.2558, grad_fn=<DivBackward0>), tensor(1.2949, 

# The search command in the MASE flow

The search flow implemented in MASE is very similar to the one that you have constructed manually, the overall flow is implemented in [search.py](../machop/chop/actions/search/search.py), the following bullet points provide you pointers to the code base.

- MaseGraph: this is the [MaseGraph](../machop/chop/passes/graph/mase_graph.py) that you have used in lab2.
- Search space: The base class is implemented in [base.py](../machop/chop/actions/search/search_space/base.py) , where in the same folder you can see a range of different supported search spaces.
- Search strategy: Similar to the search space, you can find a a base class [definition](../machop/chop/actions/search/strategies/base.py), where different strategies are also defined in the same folder.
- Runner: Different [runners](../machop/chop/actions/search/strategies/runners) can produce different metrics, they may also use `transforms` to help compute certain search metrics.

This enables one to execute the search through the MASE command line interface, remember to change the name after the `--load` option.


In [10]:
# !./ch search --config configs/examples/jsc_toy_by_type.toml --load your_pre_trained_ckpt

# !./ch search --config configs/examples/jsc_toy_by_type.toml --load ../mase_output/jsc-tiny_classification_jsc_Base/software/training_ckpts/best.ckpt --load-type pl

In this scenario, the search functionality is specified in the `toml` configuration file rather than via command-line inputs. This approach is adopted due to the multitude of configuration parameters that need to be set; encapsulating them within a single, elegant configuration file enhances reproducibility.

In `jsc_toy_by_type.toml`, the `search_space` configuration is set in `search.search_space`, the search strategy is configured via `search.strategy`. If you are not familiar with the `toml` syntax, you can read [here](https://toml.io/en/v1.0.0).

> In order to accomplish the following task, it is necessary to make direct modifications to the code base. This can be challenging within the Colab environment. **It is recommended to implement the task on a local setup and utilize Colab strictly as a server to execute the search command above.** Consider Colab as a dedicated server for this purpose.

With now an understanding of how the MASE flow work, consider the following tasks

3. Implement the brute-force search as an additional search method within the system, this would be a new search strategy in MASE.


In [11]:
# !./ch search --config configs/examples/my.toml
# !./ch search --config configs/examples/jsc_tiny_my_B.toml

4. Compare the brute-force search with the TPE based search, in terms of sample efficiency. Comment on the performance difference between the two search methods.

In [ ]:
# !./ch search --config configs/examples/jsc_tiny_my_t.toml


In [6]:
# ./ch search --config configs/examples/jsc_tiny_my_lab4.toml
# ./ch search --config configs/examples/jsc_tiny_my_lab4.toml
# ./ch search_my --config configs/examples/jsc_tiny_my_lab4.toml


/bin/bash: /home/super_monkey/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: ./ch: No such file or directory
